# Décorateurs

## Introduction

Un décorateur est n'importe quel objet Python appelable que l'on peut utiliser pour modifier une fonction ou une classe. La référence à une fonction "func" ou à une classe "C" est passée à un décorateur et ce décorateur retourne une fonction ou classe modifiées. Ces dernières contiennent généralement un appel à la fonction ou classe de départ.

## Une fonction est un objet de première classe

### Une fonction est un objet

- Une fonction est un objet de première classe référencé par une variable. 
- L'affectation de cette référence à une autre variable crée une nouvelle référence, pas une nouvelle fonction. 
- La destruction d'une de ces variables ne supprime que la variable et pas la fonction (qui peut ne plus être accessible, ce qui est un autre problème).

In [3]:
def somme(a, b):
    return a + b


ajout = somme
print(somme(1, 2), ajout(1, 2))

del somme
print(ajout(1, 2))

3 3
3


### Une fonction peut être définie dans une fonction

In [6]:
def f():
    def g():
        print("Depuis la fonction {}...".format(g.__name__))
        print("Fin du travail de {}...".format(g.__name__))

    print("Depuis la fonction {}...".format(f.__name__))
    print("Appel de {} maintenant...".format(g.__name__))
    g()
    print("Fin du travail de {}...".format(f.__name__))


f()

Depuis la fonction f...
Appel de g maintenant...
Depuis la fonction g...
Fin du travail de g...
Fin du travail de f...


**Remarque :** Dans l'exemple ci-dessus, la fonction g n'est appellable que depuis la fonction f. Sa référence appartient à l'espace de noms de f.

In [4]:
def temperature(t):
    """
    Fonction qui définit et appelle une fonction capable de convertir une
    température en degrés celsius en degrés fahrenheit.
    """

    def celsius_vers_fahrenheit(x):
        return 9 * x / 5

    resultat = "La température est de {} degrés fahrenheit !".format(
        celsius_vers_fahrenheit(t))
    return resultat


print(temperature(20))

La température est de 36.0 degrés!


**Remarque :** l'exemple ci-dessus ne présente aucun intérêt pratique, autant directement définir dans l'espace de nom principal la fonction celsius_vers_fahrenheit !

Il n'est là que pour introduire la suite...

In [7]:
def traitement_elements(liste):
    """
    Fonction qui élève au carré tous les éléments d'une liste.
    """

    def carre(x):
        return x**2

    retour = list(map(carre, liste))  # Application de carre à tous les éléments de liste
    return retour


l = [1, 2, 3, 4, 5, 6]
l_au_carre = traitement_elements(l)
print(l_au_carre)

[1, 4, 9, 16, 25, 36]


Dans l'exemple ci-dessus, on sait que la fonction carre ne sera utilisée que par la fonction traitement_elements ; inutile de « polluer » l'espace de nom principal !

*À noter que la structure précédente rend la fonction traitement_elements complètement indépendante du reste du code, ce qui est une excellente chose !*

### Une fonction peut être passée comme argument

In [9]:
def g():
    print("Depuis la fonction 'g'")
    print("Fin du travail de 'g'")


def f(fonc):
    print("Depuis la fonction 'f'")
    print("Appel de '{}' maintenant :".format(fonc.__name__))
    fonc()


f(g)

Depuis la fonction 'f'
Appel de 'g' maintenant :
Depuis la fonction 'g'
Fin du travail de 'g'


In [13]:
def celsius_vers_fahrenheit(x):
    return 9 * x / 5


def fahrenheit_vers_celsius(x):
    return 5 * x / 9


def conversion(f, t):
    print("La fonction {} a été passée à {}".format(f.__name__, conversion.__name__))
    return f(t)


print(conversion(celsius_vers_fahrenheit, 100))
print(conversion(fahrenheit_vers_celsius, 180))

La fonction celsius_vers_fahrenheit a été passée à conversion
180.0
La fonction fahrenheit_vers_celsius a été passée à conversion
100.0


In [14]:
import math


def foo(func):
    print("La fonction '{}' à été passée à ".format(func.__name__, foo.__name__))
    res = 0
    for x in [1, 2, 2.5]:
        res += func(x)
    return res


print(foo(math.sin))
print(foo(math.cos))

La fonction 'sin' à été passée à 
2.3492405557375347
La fonction 'cos' à été passée à 
-0.6769881462259364


### Une fonction peut retourner une fonction : fermeture

In [15]:
def f(a, b):
    def _f(x):
        return a * x + b
    return _f

droite_1 = f(1, 2)
droite_2 = f(2, 3)

print(droite_1(2))
print(droite_2(2))

4
7


In [8]:
def ajout_a(n):
    print("La fonction {} vient d'être appelée avec l'argument {}".format(ajout_a.__name__, n))
    def ajoute(x):
        print("La fonction {} vient d'être appelée avec l'argument {}".format(ajoute.__name__, x))
        return n + x
    return ajoute

fa = ajout_a(10)
print(fa(4))

fb = ajout_a(6)
print(fb(4))

La fonction ajout_a vient d'être appelée avec l'argument 10
La fonction ajoute vient d'être appelée avec l'argument 4
14
La fonction ajout_a vient d'être appelée avec l'argument 6
La fonction ajoute vient d'être appelée avec l'argument 4
10


## Premier décorateur

In [25]:
def decorateur(func):
    def fonction_enveloppe(x):
        print("* Avant appel de {} *".format(func.__name__))
        func(x)
        print("* Après appel de {} *".format(func.__name__))
    return fonction_enveloppe

def foo(x):
    print("La fonction foo a été appelée avec l'argument {}".format(x))

print("- Appel de foo avant toute décoration :")
foo("Hello")
    
print("- Décoration de foo avec f :")
foo = decorateur(foo)

print("- Appel de foo après décoration :")
foo(42)

- Appel de foo avant toute décoration :
La fonction foo a été appelée avec l'argument Hello
- Décoration de foo avec f :
- Appel de foo après décoration :
* Avant appel de foo *
La fonction foo a été appelée avec l'argument 42
* Après appel de foo *
